# Importing Necessary Files

In [1]:
import pandas as pd
import numpy as np

***

# Creating DataFrame(Table) of required data

**User data of ratings of movies**

In [4]:
column_names = ['user_id', 'item_id', 'rating', 'timestamp']
users = pd.read_csv('u.data', sep='\t', names=column_names)

In [6]:
users.head()

,user_id,item_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


***

**Data about movies according to their features**

In [8]:
i_cols = ['movie id', 'movie title' ,'release date','video release date', 'IMDb URL', 'unknown', 'Action', 'Adventure',
 'Animation', 'Children\'s', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy',
 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']
movies = pd.read_csv('u.item', sep='|', names=i_cols,encoding='latin-1')

In [13]:
movies.head()

,movie id,movie title,release date,video release date,IMDb URL,unknown,Action,Adventure,Animation,Children's,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


***

**Creating table of movies with their ratings using users dataframe**

In [33]:
k = pd.DataFrame(users.groupby('item_id')['rating'].mean())

In [178]:
k.head()

,rating
movie id,
1,3.878319
2,3.206107
3,3.033333
4,3.550239
5,3.302326


**Combing ratings with movies table**

In [30]:
movies.drop(['release date','video release date','IMDb URL'],axis=1,inplace=True)
k.index.names = ['movie id']
movies.set_index('movie id',inplace=True)
movies = movies.join(k)

In [183]:
movies.head()

,movie title,unknown,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,...,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,rating
movie id,,,,,,,,,,,,,,,,,,,,,
1,Toy Story (1995),0,0,0,1,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,3.878319
2,GoldenEye (1995),0,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,3.206107
3,Four Rooms (1995),0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,3.033333
4,Get Shorty (1995),0,1,0,0,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,3.550239
5,Copycat (1995),0,0,0,0,0,0,1,0,1,...,0,0,0,0,0,0,1,0,0,3.302326


***

# Comparing Movies using consine similarity

In [50]:
from sklearn.metrics.pairwise import cosine_similarity

In [189]:
movies.columns

Index(['movie title', 'unknown', 'Action', 'Adventure', 'Animation',
       'Children's', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy',
       'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi',
       'Thriller', 'War', 'Western', 'rating'],
      dtype='object')

In [193]:
movie_info = movies.drop([ 'unknown', 'Action', 'Adventure', 'Animation',
       "Children's", 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy',
       'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi',
       'Thriller', 'War', 'Western'],axis=1)

***

**Function to find similar movies**

In [646]:
def similar_movies(name):
    star_df = movies[movies['movie title'].str.contains(name)==True]

    if(len(star_df)>0):
        sim = cosine_similarity(star_df.drop(['movie title','rating'],axis=1),movies.drop(['movie title','rating'],axis=1))
        
        new_sim = np.empty(1682)
        new_sim.fill(0)

        if(len(star_df)>1):
            for i in range(len(star_df)):
                new_sim += (sim[i]/len(star_df))
        else:
            new_sim = sim
        
        new_sim = new_sim.reshape((1682,1))
        sim_df = pd.DataFrame(new_sim,columns=['Correlation'],index=range(1,1683))
        sim_df.index.name = 'movie id'
        sim_df = sim_df.join(movie_info)
        return sim_df
    return False

***

**Function to query the table**

In [647]:
def movie_n(name,correlation,number,rating):

    sim_df = similar_movies(name)
    #print(sim_df.head(2))
    if(type(sim_df)!=type(True)):
        k = sim_df[(sim_df['Correlation']>correlation) & (sim_df['rating']>rating)][['movie title','Correlation']]
        k.sort_values('Correlation',ascending=False,inplace=True)
        k.drop(['Correlation'],axis=1,inplace=True)
        if(len(k)>0):
            print(k.head(number))
        else:
            print("Sorry, movie can't be recommended with given constraint")
    else:
        print('Movie you like does not exist in database')

***

**QUERY**

In [650]:
name = input('Enter the name of movie: ')
simi = float(input('Enter pecentage of similarity (1-100): '))/100
rating = float(input('Enter minimum ratings of recommended movies: '))
number = int(input('Enter number of movies to be recommended: '))

movie_n(name,simi,number,rating)

Enter the name of movie: Star Wars
Enter pecentage of similarity (1-100): 70
Enter minimum ratings of recommended movies: 4
Enter number of movies to be recommended: 5
                              movie title
movie id                                 
50                       Star Wars (1977)
181             Return of the Jedi (1983)
172       Empire Strikes Back, The (1980)
498             African Queen, The (1951)
